The last thing I was working on was:
* modifying the advect_layers function to stop layers from going crazy as they reach the edges of the domain (hopefully done)
* adjusting the bounds around which the main ODE is solved to end when layers reach this boundary area
* adapting all the analysis code to not query the ODE solutions past where they are valid

After messing with this a bit, potentially none of this is necessary. Instead, I added a maximum step size (in time) between reinterpolating the layer tracer points. This seems to resolve the stability issues I was seeing.

The logistic basal velocity field seems more reasonable. Need to play with it some more, but 30 and 50 m/yr max basal velocity seem to both produce reasonable results. Hopefully can use layers from 30 in the 50 simulation to capture Dusty's speed up idea.

Need to figure out how to calculate du/dz under the zero slope approximation.

Decided it made more sense to just compare vertical velocities directly. Need to figure out why the vertical velocity estimate from the ODEs is consistently slightly off. Error plot has a weird pattern. Then maybe add in a "layer direction represents flow direction" assumption to test in addition to zero slope.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sympy
from sympy import *
import pickle
import datetime
from tqdm import tqdm

### Problem setup

In [ ]:
n = 2.5 # Flow exponent for the actual model

output_results_base = f"outputs/{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}_n{n:.1f}"
print(f"{output_results_base}")

In [ ]:
# Domain size
domain_x = 100000 # meters # 100 km for all other examples, 200 km for the new one
domain_z = 3000 # meters

# Grids for when discretization is needed
dx = 100
dz = 25
xs = np.arange(0, domain_x, dx)
zs = np.arange(0, domain_z, dz)

# Sympy symbolic variables
x = sympy.symbols('x', real=True, positive=True)
z = sympy.symbols('z', real=True, positive=True)

# Define surface geometry
surface_sym = domain_z - ((x / 18000.0)**3.0) # Rheology example

#surface_sym = 3000 - (((x+40000) / 25000.0)**3.0)

#surface_sym = 2500 - (((x+130000) / 30000.0)**3.0)

#surface_sym = 2000 - 1e-5*(x/domain_x)**2
#surface_sym = 2500 - (((x+40000) / 25000.0)**3.0) # Zero slope error example #1 (dtan(alpha)/dz = 0)
#surface_sym = 2500 + (1e-10*x)**2 # Zero slope error example #2 (du/dz = 0)
#surface_sym = ((domain_z-1600) / (1 + exp(0.00008*(x-(domain_x/2))))) + 1600

surface = sympy.lambdify(x, surface_sym, modules='numpy')


# Use sympy to build a function for the derivative of the surface
ds_dx_sym = sympy.diff(surface_sym, x)

ds_dx_lambdify = sympy.lambdify(x, ds_dx_sym, modules='numpy')
def ds_dx(x):
    tmp = ds_dx_lambdify(x)
    if np.isscalar(tmp):
        return tmp + (0*x)
    else:
        return tmp

# Plot the surface and its derivative with twin x axes
fig, ax = plt.subplots(figsize=(8, 4))
line_surf = ax.plot(xs/1e3, surface(xs), 'blue', label='s(x) [m]')
ax.tick_params(axis='y', colors='blue')
ax.set_ylabel('Surface elevation [m]', color='blue')
ax_right = ax.twinx()
ax_right.tick_params(axis='y', colors='red')
line_slope = ax_right.plot(xs/1e3, (180/np.pi) * np.tan(ds_dx(xs)), 'r--', label='ds_dx(s) [deg]')
ax_right.set_ylabel('Surface slope [deg]', color='red')

lns = line_surf + line_slope
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='upper right')

ax.set_title('Surface and surface slope')
ax.set_xlabel('Distance [km]')
ax.grid(True, axis='x')
fig.savefig(f"{output_results_base}_surface_and_slope.png")
plt.show()

### Generate SIA-based velocity field

This is a basic SIA velocity field, heavily based on course notes here: https://ocw.hokudai.ac.jp/wp-content/uploads/2016/02/DynamicsOfIce-2005-Note-all.pdf

The input functions (surface(x) and ds/dx(x)) are passed in as symbolic expressions and the outputs are also returned as symbolic expressions.


In [ ]:
def sia_model(x_sym, z_sym, surface_sym, dsdx_sym,
              rho=918, g=9.8, A0=3.985e-13, n_A0=3, Q=60e3, R=8.314,
              T_rel_p=(273.15-20), log_ref_stress=6,
              n=3, basal_velocity_sym=0):
    """
    Implementation is based on Section 3.4 of Ralf Greve's course notes:
    https://ocw.hokudai.ac.jp/wp-content/uploads/2016/02/DynamicsOfIce-2005-Note-all.pdf

    Constant defaults and units:
    ρ = 918.0 # kg/m^3
    g = 9.8 # m/s^2
    A0 = 3.985e-13 # s^-1 Pa^-3
    Q = 60.0e3 # J mol^-1
    R = 8.314 # J mol^-1 K^-1
    T_rel_p = 273.15 - 20 # K
    """

    # Baseline case A value
    A = A0 * np.exp(-Q / (R * T_rel_p))

    if n != n_A0:
        log_ref_strain_rate = np.log10(2*A) + n_A0 * log_ref_stress
        log_2A = log_ref_strain_rate - n * log_ref_stress # log10(2*A)
        A = 10**(log_2A) / 2
    
    #print(f"n = {n:.2f}, A = {A:.2e}")

    # Solve for u (horizontal velocity)
    # u_sym = (-2.0 * A * Abs(dsdx_sym)**(n-1.0) * dsdx_sym * rho**n * g**n *
    #          (surface_sym**(n+1.0) - Max(surface_sym - z_sym, 0)**(n+1.0)) / (n + 1.0)) + basal_velocity(x_sym)
    u_sym = (-2.0 * A * Abs(dsdx_sym)**(n-1.0) * dsdx_sym * rho**n * g**n *
             (surface_sym**(n+1.0) - (surface_sym - z_sym)**(n+1.0)) / (n + 1.0)) + basal_velocity_sym
    
    # Recover w (vertical velocity) from u through incompressibility
    du_dx_sym = sympy.diff(u_sym, x_sym)
    dw_dz_sym = -1 * du_dx_sym

    # Integrate up from the bed to find w(x, z)
    # Assume: w(x, z=0) =0 (no basal melt)
    w_sym = sympy.integrate(dw_dz_sym, (z_sym, 0, z_sym))

    return u_sym, w_sym, du_dx_sym


Our SIA model is fully specified by a surface contour (including its derivative) and a basal velocity field. (Plus some constants, but we'll assume those don't change.)

We'd like to generate multiple examples with varying rheology (in this case defined as varying flow exponents) but identical surface velocity fields. In order to do this, we run the SIA model three times:
* First with a "reference n" flow exponent (that must be the lowest value we want to use) and zero basal velocity
* Again with the intended flow exponent and zero basal velocity
* And finally with the intended flow exponent and a basal velocity field calculated to compensate for the difference between the surface velocities of the two prior models.

This means that if we change nothing except for `n` between runs, we get identical surface velocity fields but varying englacial velocities.

In [ ]:
import scipy.constants


reference_n = 2.0 # Flow exponent for the reference model -- 2.0 for all examples EXCEPT the layer dip example

assert n >= reference_n, ("The reference model flow exponent must be <= n or "
                          "else negative horizontal basal velocities are "
                          "required to make the surface velocities match.")

# Reference run
u_ref, _, _ = sia_model(x, z, surface_sym, ds_dx_sym, n=reference_n)
# Run with intended flow exponent to figure out the needed basal velocity
u_test, _, _ = sia_model(x, z, surface_sym, ds_dx_sym, n=n)
basal_velocity = u_ref.subs(z, surface_sym) - u_test.subs(z, surface_sym)
#basal_velocity = basal_velocity + 10/scipy.constants.year # Zero slope error example #2 (du/dz = 0)

#((20/scipy.constants.year) / (1 + sympy.exp(-0.0002*(x-30000)))) + ((-20/scipy.constants.year) / (1 + sympy.exp(-0.0002*(x-40000)))) + \
basal_velocity = ((20/scipy.constants.year) / (1 + sympy.exp(-0.0005*(x-75000)))) #+ ((-20/scipy.constants.year) / (1 + sympy.exp(-0.0003*(x-85000)))) # Layer dip example

# The actual model we'll use going forward
u, w, du_dx = sia_model(x, z, surface_sym, ds_dx_sym, n=n, basal_velocity_sym=basal_velocity)

# Plot the resulting horizontal and vertical velocity fields
X, Z = np.meshgrid(xs, zs)
U = sympy.lambdify((x, z), u, modules='numpy')(X, Z)
W = sympy.lambdify((x, z), w, modules='numpy')(X, Z)

below_surface = (Z < surface(X))
below_surface_mask = np.ones_like(X, dtype=float)
below_surface_mask[~below_surface] = np.nan

fig, (ax_U, ax_dWdz) = plt.subplots(2,1, figsize=(8, 6), sharex=True)
pcm_U = ax_U.pcolormesh(X/1e3, Z, scipy.constants.year*U*below_surface_mask, cmap='viridis', vmin=0)
fig.colorbar(pcm_U, ax=ax_U, label='Horizontal velocity [m/yr]')
ax_U.set_title('Horizontal velocity')
ax_U.set_ylabel('z [m]')
ax_U.set_ylim(0, domain_z)
pcm_W = ax_dWdz.pcolormesh(X/1e3, Z, scipy.constants.year*W*below_surface_mask, cmap='viridis')
fig.colorbar(pcm_W, ax=ax_dWdz, label='Vertical velocity [m/yr]')
ax_dWdz.set_title('Vertical velocity')
ax_dWdz.set_xlabel('x [km]')
ax_dWdz.set_ylabel('z [m]')
fig.tight_layout()
fig.savefig(f"{output_results_base}_velocity_fields.png")
plt.show()

In [ ]:
# For verification purposes, also plot the surface velocity

# Calculate the surface velocity (just below the surface to avoid border effects)
u_surface = sympy.lambdify(x, u.subs(z, surface_sym-0.1), modules='numpy')(xs)
u_bed = sympy.lambdify(x, u.subs(z, 0.1), modules='numpy')(xs)

# Plot the surface velocity
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xs/1e3, scipy.constants.year*u_surface, 'k-', label='Surface velocity [m/yr]')
ax.plot(xs/1e3, scipy.constants.year*u_bed, 'r-', label='Basal velocity [m/yr]')
#ax.set_title('Surface velocity')
ax.legend()
ax.set_xlabel('Distance [km]')
ax.set_ylabel('Horizontal Velocity [m/yr]')
ax.grid(True)
fig.savefig(f"{output_results_base}_surface_bed_velocity.png")
plt.show()

In [ ]:
# Vertical velocity at the bed

w_bed = sympy.lambdify(x, w.subs(z, 0.1), modules='numpy')(xs)

# Plot the surface velocity
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(xs/1e3, scipy.constants.year*w_bed, label='Basal velocity [m/yr]')
ax.legend()
ax.set_xlabel('Distance [km]')
ax.set_ylabel('Vertical Velocity [m/yr]')
ax.grid(True)
plt.show()

### Generate synthetic layers

In [ ]:
def advect_layer(u, w, xs, initial_layer, layer_ages, xs_initial=None, max_age_timestep=None):
    """
    Advect a layer in a vertical velocity field w and horizontal velocity field u.
    The layer is represented as a 1D array of thicknesses at each x position.

    Advection is done by solving an ODE for sampled particles at locations (xs, initial_layer(xs))
    moving through the velocity field (u, w). The ODE is solved for each layer age.

    Args:
    u: Function mapping (x, z) to horizontal velocity at that position
    w: Function mapping (x, z) to vertical velocity at that position
    xs: 1D array of x positions
    initial_layer: A function mapping x positions to the initial layer position
    layer_ages: 1D array of ages of each layer to be output
    xs_initial: Optional 1D array of x positions to use for the initial layer positions. If None, use xs.

    Returns:
    An array (of the same length as layer_ages) of functions mapping x positions to the layer position at that age
    """

    # Use the same x positions for the initial layer as the output x positions if not specified
    if xs_initial is None:
        xs_initial = xs

    zs_initial = initial_layer(xs_initial)

    # Encode x and y coordinates into flattened list for scipy.integrate.solve_ivp
    y = np.concatenate([xs_initial, zs_initial])

    # Simple 2D advection ODE
    def ode_fun(t, y):
        x = y[:len(y)//2]
        z = y[len(y)//2:]
        us = u(x, z)
        ws = w(x, z)

        dydt = np.concatenate([us, ws])
        return dydt
    
    # Integrate the ODE for each layer age
    # Need to do it this way rather than solving for all ages at once because
    # points move too far out of the domain. Re-interpolating them back onto
    # normal x spacing between time steps fixes this.
    t = 0
    advected_layers = []
    for idx, age in enumerate(layer_ages):
        if max_age_timestep is not None:
            while (age-t) > max_age_timestep:
                #print(f"Requested next step age: {age}, delta: {age - t}, actually taking step of size {max_age_timestep}")
                sol = scipy.integrate.solve_ivp(ode_fun, [0, max_age_timestep], y, dense_output=True, rtol=1e-8, atol=1e-8)
                xs_advected = sol.y[:len(y)//2, -1]
                zs_advected = sol.y[len(y)//2:, -1]
                
                #layer_interp_for_next_layer = scipy.interpolate.interp1d(xs_advected, zs_advected, fill_value='extrapolate')

                sorted_indices = np.argsort(xs_advected)
                layer_interp_for_next_layer = scipy.interpolate.PchipInterpolator(xs_advected[sorted_indices], zs_advected[sorted_indices])
                
                y = np.concatenate([xs, layer_interp_for_next_layer(xs)])
                t += max_age_timestep

        sol = scipy.integrate.solve_ivp(ode_fun, [0, age-t], y, dense_output=True, rtol=1e-8, atol=1e-8)
        xs_advected = sol.y[:len(y)//2, -1]
        zs_advected = sol.y[len(y)//2:, -1]
        
        #layer_interp_for_next_layer = scipy.interpolate.interp1d(xs_advected, zs_advected, fill_value='extrapolate')

        sorted_indices = np.argsort(xs_advected)
        layer_interp_for_next_layer = scipy.interpolate.PchipInterpolator(xs_advected[sorted_indices], zs_advected[sorted_indices])
        
        y = np.concatenate([xs, layer_interp_for_next_layer(xs)])
        t = age

        layer_interp = layer_interp_for_next_layer

        advected_layers.append(layer_interp)

    return advected_layers


In [ ]:
xs_layers = np.linspace(0, domain_x, 200) # don't need tight grid spacing for smooth layers

# For advecting layers: don't advect from surface locations where the vertical velocity is positive (ablation zone)
# w_fun = sympy.lambdify((x, z), w, modules='numpy')
# xs_layers_initial = xs_layers[w_fun(xs_layers, surface(xs_layers))*scipy.constants.year < -1]
xs_layers_initial = xs_layers

# As they were made in the Julia script
# layer_ages = np.logspace(2, 5, 25)
# layers_t0 = advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
#                          sympy.lambdify((x, z), w, modules='numpy'),
#                          xs_layers, lambda x: surface(x) - 200, layer_ages*scipy.constants.year,
#                          xs_initial=xs_layers_initial,
#                          max_age_timestep=50*scipy.constants.year)

# With a little spacing between them
layer_ages = np.logspace(0, 3, 28)
layers_t0 = []
for idx, age in enumerate(layer_ages):
    start_offset = 100 + (idx * 100)
    layer_start_fn = lambda x: surface(x) - start_offset
    layer = advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
                         sympy.lambdify((x, z), w, modules='numpy'),
                         xs_layers, layer_start_fn, layer_ages[:idx+1]*scipy.constants.year,
                         max_age_timestep=100*scipy.constants.year)
    layers_t0.append(layer[-1])

# Layers from bottom starting flat and conforming to surface -- Used for rheology example
# layers_t0 = []
# for idx, start_offset in enumerate(np.arange(500, 2900, 200)):
#     layer_start_fn = lambda x: start_offset + (idx/10)*(surface(x)-surface(0))
#     layer = advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
#                          sympy.lambdify((x, z), w, modules='numpy'),
#                          xs_layers, layer_start_fn, [scipy.constants.year * 0])
#     layers_t0.append(layer[-1])

# Manually specified layers -- Sloping layers example
# layers_t0 = []
# layer_base_fn = lambda x: ((2200-1900) / (1 + np.exp(0.00015*(x-(2*domain_x/3)))))
# start_offsets = np.arange(-2400, 0, 150)
# for idx, start_offset in enumerate(start_offsets):
#     layer_start_fn = lambda x: (idx/len(start_offsets))*layer_base_fn(x) - start_offset + (1 - idx/len(start_offsets))*(surface(x)-surface(0))
#     layer = advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
#                          sympy.lambdify((x, z), w, modules='numpy'),
#                          xs_layers, layer_start_fn, [scipy.constants.year * 0])
#     layers_t0.append(layer[-1])

# Manually specified layers -- Zero slope error example #1 (dtan(alpha)/dz = 0)
# layers_t0 = []
# layer_base_fn = lambda x: ((2200-1700) / (1 + np.exp(0.0002*(x-(2*domain_x/3))))) + 1600
# for idx, start_offset in enumerate(np.arange(0, 1500, 150)):
#     layer_start_fn = lambda x: (1 + 0*idx)*layer_base_fn(x) - start_offset
#     layer = advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
#                          sympy.lambdify((x, z), w, modules='numpy'),
#                          xs_layers, layer_start_fn, [scipy.constants.year * 0])
#     layers_t0.append(layer[-1])

# # Manually specified layers -- Zero slope error example #2 (du/dz = 0)
# layers_t0 = []
# layer_base_fn = lambda x: ((2200-1700) / (1 + np.exp(0.0002*(x-(2*domain_x/3))))) + 1600
# for idx, start_offset in enumerate(np.arange(2000, 500, -150)):
#     def layer_fn(x):
#         x = x - domain_x/2
#         l = -300 * idx/10 * np.exp(-1 / (1-(x/20e3)**2))
#         l[np.abs(x) >= 20e3] = 0
#         l += start_offset
#         return l
#     layer = advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
#                          sympy.lambdify((x, z), w, modules='numpy'),
#                          xs_layers, layer_fn, [scipy.constants.year * 0])
#     layers_t0.append(layer[-1])

In [ ]:
# Plot the advected layers on top of a pcolormesh plot of the velocity magnitude
X, Z = np.meshgrid(xs, zs)
U = sympy.lambdify((x, z), u, modules='numpy')(X, Z)
W = sympy.lambdify((x, z), w, modules='numpy')(X, Z)
vel = np.sqrt(U**2 + W**2)

fig, ax = plt.subplots(figsize=(8, 4))
pcm = ax.pcolormesh(X/1e3, Z, scipy.constants.year * vel * below_surface_mask, cmap='viridis', alpha=0.8) #vmax=170)
fig.colorbar(pcm, ax=ax, label='Velocity magnitude [m/yr]')
for layer in layers_t0:
    ax.plot(xs_layers/1e3, layer(xs_layers), 'k--')
# for layer in layers_t1:
#     ax.plot(xs_layers/1e3, layer(xs_layers), 'r--')
#ax.set_title('Velocity magnitude and advected layers')
ax.set_title(f'n = {n}')
ax.grid(True)
ax.set_xlim(0, domain_x/1e3)
ax.set_ylim(0, surface(0))
ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')
fig.savefig(f"{output_results_base}_layers.png")
plt.show()

### Advect layers by 1 year to simulate layer motion

In [ ]:
xs_layers_t1 = np.arange(0, domain_x, 10)
layers_t1 = []
for layer in layers_t0:
    layers_t1.append(advect_layer(sympy.lambdify((x, z), u, modules='numpy'),
                                  sympy.lambdify((x, z), w, modules='numpy'),
                                  xs_layers_t1, layer, [scipy.constants.year * 1])[-1])

In [ ]:
layer_idx = 15
xs_layers_t1 = np.arange(0, domain_x, 10)

fig, ax = plt.subplots(figsize=(8, 4))
#ax.plot(xs_layers/1e3, layers_t0[layer_idx](xs_layers), 'k-', label='t=0')
# ax.plot(xs_layers/1e3, layers_t1[layer_idx](xs_layers), 'r-', label='t=1')
ax.plot(xs_layers_t1/1e3, layers_t1[layer_idx](xs_layers_t1) - layers_t0[layer_idx](xs_layers_t1), 'r-', label='t1 - t0')
ax.set_title('Layer advection')
ax.legend()
ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')
ax.grid(True)
# ax.set_xlim(0, 3)
# ax.set_ylim(1820, 1860)

### Set up finite difference approximations of layer deformation-related partial derivatives

In [ ]:
# snr_linear = 10**(3/20) # diml
# prf = 1000 # Hz
# velocity = 50 # m/s
# center_frequency = 60e6 # Hz
# dielectric_constant = 3.17


# pulse_spacing = velocity / prf # m
# pulses_x = np.arange(0, domain_x, pulse_spacing)

# rng = np.random.default_rng(275209073368752189122200994498511502265)
# ph_noise = rng.normal(0, np.sqrt(1/snr_linear), (len(pulses_x), len(layers_t0), 2)) # shape: (pulses, layers, t0/t1)
# layer_position_noise = scipy.constants.c * ph_noise / (center_frequency * np.sqrt(dielectric_constant) * 4 * np.pi)

# layer_position_noise_interp = {0: {}, 1: {}}
# layers_t0_noise = []
# layers_t1_noise = []
# for idx in range(len(layers_t0)):
#     layer_position_noise_interp[0][idx] = scipy.interpolate.interp1d(pulses_x, layer_position_noise[:, idx, 0], fill_value='extrapolate', kind='nearest')
#     layer_position_noise_interp[1][idx] = scipy.interpolate.interp1d(pulses_x, layer_position_noise[:, idx, 1], fill_value='extrapolate', kind='nearest')
#     layers_t0_noise.append(lambda x, idx=idx: layers_t0[idx](x) + layer_position_noise_interp[0][idx](x))
#     layers_t1_noise.append(lambda x, idx=idx: layers_t1[idx](x) + layer_position_noise_interp[1][idx](x))

# # Define layer functions with noise

# #layer_deformation_noise

In [ ]:
def layer_dl_dx(x, layer_idx, dx=1):
    """
    Numerical central difference approximation of layer slope
    Output units are m/m
    """
    return (layers_t0[layer_idx](x+(dx/2)) - layers_t0[layer_idx](x-(dx/2))) / dx

def layer_dl_dt(x, layer_idx):
    """
    Numerical forward difference approximation of layer vertical deformation
    Output units are m/year
    """
    return (layers_t1[layer_idx](x) - layers_t0[layer_idx](x))

def layer_d2l_dxdz(x, layer_idx):
    """
    Numerical central difference approximation of d^2l/(dxdz)
    Output units are m/m^2
    """
    layer_p1 = layer_dl_dx(x, layer_idx+1)
    layer_m1 = layer_dl_dx(x, layer_idx-1)
    layer_dz = layers_t0[layer_idx+1](x) - layers_t0[layer_idx-1](x)
    return (layer_p1 - layer_m1) / (layer_dz)

def layer_d2l_dtdz(x, layer_idx):
    """
    Numerical central difference approximtion of d^2l/(dtdz)
    Output units are m/(m*year)
    """
    layer_p1 = layer_dl_dt(x, layer_idx+1)
    layer_m1 = layer_dl_dt(x, layer_idx-1)
    layer_dz = layers_t0[layer_idx+1](x) - layers_t0[layer_idx-1](x)
    return (layer_p1 - layer_m1) / (layer_dz)


In [ ]:
# Plot the advected layers on top of a pcolormesh plot of the velocity magnitude
X, Z = np.meshgrid(xs, zs)
U = sympy.lambdify((x, z), u, modules='numpy')(X, Z)
W = sympy.lambdify((x, z), w, modules='numpy')(X, Z)
vel = np.sqrt(U**2 + W**2)

fig, ax = plt.subplots(figsize=(8, 4))
#pcm = ax.pcolormesh(X/1e3, Z, scipy.constants.year * vel * below_surface_mask, cmap='grey', alpha=0.5)
vmin, vmax = -2, 2
sc = None
for idx, layer in enumerate(layers_t0):
    sc = ax.scatter(xs/1e3, layer(xs), c=np.arctan(layer_dl_dx(xs, idx))*(180/np.pi), s=2, vmin=vmin, vmax=vmax, cmap='coolwarm')
fig.colorbar(sc, ax=ax, label='Layer slope [deg]')
ax.set_title('Layer slope')
ax.grid(True)
ax.set_xlim(0, domain_x/1e3)
ax.set_ylim(0, surface(0))
ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')
fig.savefig(f"{output_results_base}_layer_slope.png")
plt.show()

### Method of Characteristics Solution

In [ ]:
def du_dtau(tau, u, layer_idx):
    res = -1 * layer_d2l_dxdz(tau, layer_idx)*u - layer_d2l_dtdz(tau, layer_idx)
    #res[np.where(u < 0)] = np.max(res, 0)
    return res

start_pos_x = 100

layer_solutions = {}

for idx in tqdm(np.arange(1, len(layers_t0)-1)):
    layer = layers_t0[idx]
    u0 = u.subs([(x, start_pos_x), (z, layer(start_pos_x))]).evalf() * scipy.constants.year
    # If you're extracting points from the solution (for rheology estimates, for example), set max_step to no more than the spacing at which you'll extract points
    # (but if you're just plotting velocity, this will run a lot faster if you let the solver pick a max step size)
    in_lower_guardrails = xs_layers[layer(xs_layers) < 200]
    if len(in_lower_guardrails) > 0:
        layer_end = np.min(in_lower_guardrails)
    else:
        layer_end = domain_x
    layer_solutions[idx] = scipy.integrate.solve_ivp(du_dtau, [start_pos_x, layer_end], np.array([u0]), args=(idx,), dense_output=True, max_step=100)
    if layer_solutions[idx].status != 0:
        print(f"Layer {idx} failed to solve")
        print(layer_solutions[idx].message)

In [ ]:
def layer_solution_velocity(layer_idx, x):
    res = layer_solutions[layer_idx].sol(x)[0]
    if np.isscalar(res):
        if x > layer_solutions[layer_idx].sol.t_max:
            return np.nan
        else:
            return res
    res[x > layer_solutions[layer_idx].sol.t_max] = np.nan
    return res

In [ ]:
fig, (ax, ax_err) = plt.subplots(2, 1, figsize=(8, 8), sharex=True, sharey=True)
sc = None

# Solution plot
for layer_idx in layer_solutions.keys():
    xs_tmp = xs
    sc = ax.scatter(xs_tmp/1e3, layers_t0[layer_idx](xs_tmp), c=layer_solution_velocity(layer_idx, xs_tmp), vmin=0, vmax=40, s=2, cmap='viridis')
fig.colorbar(sc, ax=ax, label='Horizontal Velocity [m/yr]')

ax.grid(True)
ax.set_xlim(0, domain_x/1e3)
ax.set_ylim(0, surface(0))
#ax.set_xlabel('x [km]')
ax.set_ylabel('z [m]')

# Error plot

for layer_idx in layer_solutions.keys():
    xs_tmp = xs
    err = layer_solution_velocity(layer_idx, xs_tmp) - (sympy.lambdify((x,z), u, modules='numpy')(xs_tmp, layers_t0[layer_idx](xs_tmp)) * scipy.constants.year)
    sc = ax_err.scatter(xs_tmp/1e3, layers_t0[layer_idx](xs_tmp), c=err, vmin=-2, vmax=2, s=2, cmap='coolwarm_r')
fig.colorbar(sc, ax=ax_err, label='Horizontal Velocity Error [m/yr]')

ax_err.grid(True)
ax_err.set_xlabel('x [km]')
ax_err.set_ylabel('z [m]')

ax.set_title('Horizontal velocity solution')
ax_err.set_title('Error in horizontal velocity estimate')
fig.savefig(f"{output_results_base}_layer_solutions.png")
plt.show()

In [ ]:
# Plot each layer solution at x=100e3 as a function of depth
fig, (ax_u, ax_dwdz) = plt.subplots(1,2, figsize=(8, 8), sharey=True)

plot_pos_x = 80e3

u_at_plot_pos = sympy.lambdify(z, u.subs(x, plot_pos_x), modules='numpy')(zs)

for layer_idx in layer_solutions.keys():
    if layer_idx == 1:
        lbl = 'ODE Solutions'
    else:
        lbl = None
    ax_u.scatter([layer_solution_velocity(layer_idx, plot_pos_x)], [layers_t0[layer_idx](plot_pos_x)], label=lbl, c='r')

ax_u.plot(u_at_plot_pos*scipy.constants.year, zs, 'k--', label='True')
ax_u.set_title(f'Horizontal Velocity\nn = {n}')
ax_u.set_xlabel('Horizontal velocity [m/yr]')
ax_u.set_ylabel('z [m]')
ax_u.grid()
ax_u.legend()
ax_u.set_xlim(0, 1.2 * np.nanmax(u_at_plot_pos*scipy.constants.year))

# Vertical strain rate
dwdz_at_plot_pos = sympy.lambdify(z, sympy.diff(w, z).subs(x, plot_pos_x), modules='numpy')(zs)

for layer_idx in layer_solutions.keys():
    if layer_idx == 1:
        lbl_ode = 'ODE Solutions'
        lbl_zeroapprox = 'Zero Slope Approximation'
    else:
        lbl_ode = None
        lbl_zeroapprox = None
    
    # Recovered from layer ODE solutions
    du_dx_ode_at_plot_pos = layer_solution_velocity(layer_idx, plot_pos_x) - layer_solution_velocity(layer_idx, plot_pos_x-1)
    ax_dwdz.scatter([-1*du_dx_ode_at_plot_pos], [layers_t0[layer_idx](plot_pos_x)], label=lbl_ode, c='r')

    # Estimated from zero slope approximation
    dwdz_zs_approx = (layer_dl_dt(plot_pos_x, layer_idx+1) - layer_dl_dt(plot_pos_x, layer_idx-1)) / (layers_t0[layer_idx+1](plot_pos_x) - layers_t0[layer_idx-1](plot_pos_x))
    ax_dwdz.scatter([dwdz_zs_approx], [layers_t0[layer_idx](plot_pos_x)], label=lbl_zeroapprox, c='b', marker='x')

ax_dwdz.plot(dwdz_at_plot_pos*scipy.constants.year, zs, 'k--', label='True')
ax_dwdz.set_title(f'Vertical Strain Rate\nn = {n}')
ax_dwdz.set_xlabel('Vertical strain rate [m/(yr*m)]')
ax_dwdz.grid()
ax_dwdz.legend()

fig.savefig(f"{output_results_base}_layer_solutions_at_xpos.png")
plt.show()

### Estimate effective stress

In [ ]:
# Estimate du_dz
# dudz_central_diff = (layer_sols[layer_idx-1](x_pos) - layer_sols[layer_idx+1](x_pos)) / ( seconds_per_year * (layers_t0[layer_idx-1](x_pos)[1] - layers_t0[layer_idx+1](x_pos)[1]))

# # Estimate effective stress under SIA
# dist_to_surf = surface(x_pos) - z_pos
# rheology_eff_stress = ρ * g * dist_to_surf * -dsdx(x_pos) # (3.88)

In [ ]:
rho = 918
g = 9.8

visc_start_x = 5e3
visc_end_x = 95e3

xs_visc = np.linspace(visc_start_x, visc_end_x, 100)

# Estimate du_dz and effective viscosity for the entire domain
du_dz_central_diff = np.zeros((len(xs_visc), len(layer_solutions)-1))
eff_stress = np.zeros_like(du_dz_central_diff)

for idx, x_pos in enumerate(xs_visc):
    for layer_idx in np.arange(2, len(layers_t0)-2):
        du_dz_central_diff[idx, layer_idx-1] = (layer_solution_velocity(layer_idx-1, x_pos) - layer_solution_velocity(layer_idx+1, x_pos)) / (layers_t0[layer_idx-1](x_pos) - layers_t0[layer_idx+1](x_pos))
        dist_to_surf = surface(x_pos) - layers_t0[layer_idx](x_pos)
        eff_stress[idx, layer_idx-1] = rho * g * dist_to_surf * -1 * ds_dx(x_pos)

# Plot a scatter plot of log10(du_dz_central_diff) vs log10(eff_stress)
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(np.log10(eff_stress), np.log10(du_dz_central_diff), s=2, label=f'n = {n}')
ax.set_aspect('equal')
ax.set_xlabel('log(effective stress)')
ax.set_ylabel('log(strain rate)')
ax.legend()
ax.grid()
fig.savefig(f"{output_results_base}_stress_strain_rate.png")
plt.show()

### Save results to a pickle

In [ ]:
# Create filename containing current timestamp and n value
filename = output_results_base + ".pickle"
with open(filename, 'wb') as f:
    pickle.dump({
        'n': n,
        'xs': xs,
        'zs': zs,
        'domain_x': domain_x,
        'domain_z': domain_z,
        'surface': surface_sym,
        'x': x,
        'z': z,
        'u': u,
        'w': w,
        'ds_dx': ds_dx_sym,
        'layers_t0': layers_t0,
        'layers_t1': layers_t1,
        'layer_solutions': layer_solutions,
        'layer_solution_velocity': layer_solution_velocity,
        'eff_stress': eff_stress,
        'du_dz_central_diff': du_dz_central_diff
    }, f)

print(f"Saved to {filename}")

### Find vertical velocity from horizontal velocity

In [ ]:
# Zero slope approximation vertical strain rate

xs_tmp = xs[xs > start_pos_x]

interp_x = np.zeros((len(layer_solutions), len(xs_tmp)))
interp_z = np.zeros_like(interp_x)
interp_dl_dt = np.zeros_like(interp_x)
interp_dl_dx = np.zeros_like(interp_x)
for idx, layer_idx in enumerate(layer_solutions.keys()):
    interp_x[idx, :] = xs_tmp
    interp_z[idx, :] = layers_t0[layer_idx](xs_tmp)
    interp_dl_dt[idx, :] = layer_dl_dt(xs_tmp, layer_idx)
    interp_dl_dx[idx, :] = layer_dl_dx(xs_tmp, layer_idx)

X, Z = np.meshgrid(xs_tmp, zs)
dl_dt_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()), interp_dl_dt.flatten(), (X, Z), method='linear')
dl_dx_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()), interp_dl_dx.flatten(), (X, Z), method='linear')
dw_dz_zeroslope_grid = np.gradient(dl_dt_grid, zs, axis=0)


In [ ]:
# Interpolate layer solutions to a grid defined by xs and zs
xs_tmp = xs[xs > start_pos_x]
interp_x = np.zeros((len(layer_solutions), len(xs_tmp)))
interp_z = np.zeros_like(interp_x)
interp_u = np.zeros_like(interp_x)
interp_w = np.zeros_like(interp_x)
for idx, layer_idx in enumerate(layer_solutions.keys()):
    interp_x[idx, :] = xs_tmp
    interp_z[idx, :] = layers_t0[layer_idx](xs_tmp)
    interp_u[idx, :] = layer_solution_velocity(layer_idx, xs_tmp)

    interp_w[idx, :] = layer_dl_dt(xs_tmp, layer_idx) + interp_u[idx,:]*layer_dl_dx(xs_tmp, layer_idx)

# Gridded horizontal velocity from layer line solutions
X, Z = np.meshgrid(xs_tmp, zs)
u_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                        interp_u.flatten(), (X, Z), method='linear')
w_mol_grid = scipy.interpolate.griddata((interp_x.flatten(), interp_z.flatten()),
                                        interp_w.flatten(), (X, Z), method='linear')

# Mask inside non-outer layers
mask = np.nan * np.zeros_like(X)
l1 = layers_t0[1](xs_tmp)
l2 = layers_t0[-2](xs_tmp)
mask[Z < np.maximum(l1, l2)] = 1
mask[Z <= np.minimum(l1, l2)] = np.nan

# Vertical strain rate
dw_dz_mol_grid = -1 * np.gradient(u_mol_grid, xs_tmp, axis=1)

fig, axs = plt.subplots(5,2, figsize=(16, 12), sharex=True)
((ax_U, ax_U_err), (ax_W, ax_W_err), (ax_W_zeroslope, ax_W_zeroslope_err), (ax_dWdz, ax_dWdz_err), (ax_dWdz_zeroslope, ax_dWdz_zeroslope_err)) = axs

err_clb_pct_of_max = 0.05

# Horizontal velocity

pcm_U = ax_U.pcolormesh(X/1e3, Z, mask * u_mol_grid, cmap='viridis')
fig.colorbar(pcm_U, ax=ax_U, label='Horizontal velocity [m/yr]')
ax_U.set_title('Horizontal velocity\n(interpolated from layer ODEs)')
ax_U.set_ylabel('z [m]')

U_tmp = sympy.lambdify((x, z), u, modules='numpy')(X, Z)
pcm_U_err = ax_U_err.pcolormesh(X/1e3, Z, mask * (u_mol_grid - U_tmp*scipy.constants.year), cmap='coolwarm',
                                vmin=-1*err_clb_pct_of_max*np.max(U_tmp*scipy.constants.year),
                                vmax=err_clb_pct_of_max*np.max(U_tmp*scipy.constants.year))
fig.colorbar(pcm_U_err, ax=ax_U_err, label='Error in horizontal velocity [m/yr]')
ax_U_err.set_title('Error in horizontal velocity\n(ODE interpolation - true)')

# Vertical velocity

pcm_W = ax_W.pcolormesh(X/1e3, Z, mask * w_mol_grid, cmap='viridis')
fig.colorbar(pcm_W, ax=ax_W, label='Vertical velocity [m/yr]')
ax_W.set_title('Vertical velocity\n(interpolated from layer ODEs)')
ax_W.set_ylabel('z [m]')

W_tmp = sympy.lambdify((x, z), w, modules='numpy')(X, Z)
pcm_W_err = ax_W_err.pcolormesh(X/1e3, Z, mask * (w_mol_grid - W_tmp*scipy.constants.year), cmap='coolwarm',
                                vmin=-1*err_clb_pct_of_max*np.max(np.abs(W_tmp*scipy.constants.year)),
                                vmax=err_clb_pct_of_max*np.max(np.abs(W_tmp*scipy.constants.year)))
fig.colorbar(pcm_W_err, ax=ax_W_err, label='Error in vertical velocity [m/yr]')
ax_W_err.set_title('Error in vertical velocity\n(ODE interpolation - true)')

# Vertical velocity (zero layer slope approximation)

vmin_ref, vmax_ref = pcm_W.get_clim()
pcm_W_zeroslope = ax_W_zeroslope.pcolormesh(X/1e3, Z, mask * dl_dt_grid, cmap='viridis', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_W_zeroslope, ax=ax_W_zeroslope, label='Vertical velocity [m/yr]')
ax_W_zeroslope.set_title('Vertical velocity\n(zero slope approximation)')
ax_W_zeroslope.set_ylabel('z [m]')

vmin_ref, vmax_ref = pcm_W_err.get_clim()
pcm_W_zeroslope_err = ax_W_zeroslope_err.pcolormesh(X/1e3, Z, mask * (dl_dt_grid - W_tmp*scipy.constants.year), cmap='coolwarm', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_W_zeroslope_err, ax=ax_W_zeroslope_err, label='Error in vertical velocity [m/yr]')
ax_W_zeroslope_err.set_title('Error in vertical velocity\n(zero slope approximation - true)')

# Vertical strain rate

dwdz_abs_max = np.nanmax(np.abs(mask * dw_dz_mol_grid))
pcm_dWdz = ax_dWdz.pcolormesh(X/1e3, Z, mask * dw_dz_mol_grid, cmap='coolwarm', vmin=-1*dwdz_abs_max, vmax=dwdz_abs_max)
fig.colorbar(pcm_dWdz, ax=ax_dWdz, label='Vertical strain rate [m/(yr*m)]')
ax_dWdz.set_title('Vertical strain rate\n(interpolated from layer ODEs)')
ax_dWdz.set_xlabel('x [km]')
ax_dWdz.set_ylabel('z [m]')

dw_dz_sym = sympy.diff(scipy.constants.year * w, z)
dw_dz_lambdify = sympy.lambdify((x, z), dw_dz_sym, modules='numpy')
dw_dz_true = dw_dz_lambdify(X, Z)

vmin_ref, vmax_ref = pcm_dWdz.get_clim()
dwdz_abs_max = np.maximum(np.abs(vmin_ref), np.abs(vmax_ref))
pcm_dWdz_err = ax_dWdz_err.pcolormesh(X/1e3, Z, mask * (dw_dz_mol_grid - dw_dz_true), cmap='coolwarm', vmin=-0.1*dwdz_abs_max, vmax=0.1*dwdz_abs_max)
fig.colorbar(pcm_dWdz_err, ax=ax_dWdz_err, label='Error in vertical strain rate [m/(yr*m)]')
ax_dWdz_err.set_title('Error in vertical strain rate\n(ODE interpolation - true)')
ax_dWdz_err.set_xlabel('x [km]')
ax_dWdz_err.set_ylabel('z [m]')

# Vertical strain rate (zero slope approximation)

vmin_ref, vmax_ref = pcm_dWdz.get_clim()
pcm_dWdz_zeroslope = ax_dWdz_zeroslope.pcolormesh(X/1e3, Z, mask * dw_dz_zeroslope_grid, cmap='coolwarm', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_dWdz_zeroslope, ax=ax_dWdz_zeroslope, label='Vertical strain rate [m/(yr*m)]')
ax_dWdz_zeroslope.set_title('Vertical strain rate\n(zero slope approximation)')
ax_dWdz_zeroslope.set_xlabel('x [km]')
ax_dWdz_zeroslope.set_ylabel('z [m]')

vmin_ref, vmax_ref = pcm_dWdz_err.get_clim()
pcm_dWdz_zeroslope_err = ax_dWdz_zeroslope_err.pcolormesh(X/1e3, Z, mask * (dw_dz_zeroslope_grid - dw_dz_true), cmap='coolwarm', vmin=vmin_ref, vmax=vmax_ref)
fig.colorbar(pcm_dWdz_zeroslope_err, ax=ax_dWdz_zeroslope_err, label='Error in vertical strain rate [m/(yr*m)]')
ax_dWdz_zeroslope_err.set_title('Error in vertical strain rate\n(zero slope approximation - true)')
ax_dWdz_zeroslope_err.set_xlabel('x [km]')
ax_dWdz_zeroslope_err.set_ylabel('z [m]')

# Draw layer lines on all plots for reference
for layer in layers_t0:
    for ax in axs.flatten():
        ax.plot(xs_layers/1e3, layer(xs_layers), 'k--', linewidth=0.5)

fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(16, 4), sharex=True)
ax_U, ax_U_pct_surf = axs

# Horizontal velocity

pcm_U = ax_U.pcolormesh(X/1e3, Z, mask * u_mol_grid, cmap='viridis')
fig.colorbar(pcm_U, ax=ax_U, label='Horizontal velocity [m/yr]')
ax_U.set_title('Horizontal velocity\n(interpolated from layer ODEs)')
ax_U.set_ylabel('z [m]')

surface_u = sympy.lambdify(x, u.subs(z, surface_sym-0.1), modules='numpy')(xs_tmp) * scipy.constants.year

pcm_U_pct_surf = ax_U_pct_surf.pcolormesh(X/1e3, Z, mask * u_mol_grid / surface_u, cmap='viridis', vmax=1, vmin=0.6)
fig.colorbar(pcm_U_pct_surf, ax=ax_U_pct_surf, label='Horizontal velocity / Surface velocity [diml]')


# Draw layer lines on all plots for reference
for layer in layers_t0:
    for ax in axs.flatten():
        ax.plot(xs_layers/1e3, layer(xs_layers), 'k--', linewidth=0.5)

fig.tight_layout()
plt.show()

### Explore a single layer solution

In [ ]:
layer_idx = 13

fig, axs = plt.subplots(4, 1, figsize=(10, 12), sharex=True)
ax_U, ax_W, ax_deriv, ax_stab = axs

sol = layer_solutions[layer_idx]

# Horizontal Velocity
ax_U.set_title(f'Horizontal Velocity at Layer {layer_idx}')
ax_U.scatter(sol.t/1e3, sol.y, s=2, label=f'ODE solution for layer {layer_idx}', c='red')
# truth
xs_tmp = sol.t
ax_U.plot(xs_tmp/1e3, sympy.lambdify((x,z), u, modules='numpy')(xs_tmp, layers_t0[layer_idx](xs_tmp)) * scipy.constants.year, 'k--', alpha=0.5, label='True')

# Vertical Velocity
ax_W.set_title(f'Vertical Velocity at Layer {layer_idx}')
ax_W.scatter(sol.t/1e3, layer_dl_dt(sol.t, layer_idx) + sol.y[0,:]*layer_dl_dx(sol.t, layer_idx), s=2, label='w(x)')
ax_W.plot(xs_tmp/1e3, sympy.lambdify((x,z), w, modules='numpy')(xs_tmp, layers_t0[layer_idx](xs_tmp)) * scipy.constants.year, 'k--', alpha=0.5, label='True')

ax_W.scatter(sol.t/1e3, layer_dl_dt(sol.t, layer_idx), s=2, label='dl/dt')
ax_W.scatter(sol.t/1e3, sol.y[0,:]*layer_dl_dx(sol.t, layer_idx), s=2, label='u * dl/dx')

# Plot the derivative
ax_deriv.set_title(f'du/dtau at Layer {layer_idx}')
xs_tmp = sol.t
ax_deriv.scatter(xs_tmp/1e3, du_dtau(xs_tmp, sol.y[0,:], layer_idx), s=2, label="du_dtau(x) (ODE Function)")

# Stability criterion
ax_stab.set_title(f'd2l_dxdz at Layer {layer_idx} (stability)')
stab_crit = layer_d2l_dxdz(sol.t, layer_idx)
ax_stab.scatter(sol.t/1e3, stab_crit, s=2)

# Properties for all plots
for ax in axs:
    ax.grid(True)
    ax.legend()

plt.show()